Introduction to ORMs and Jupyter Notebooks
==========================================

### Authors:
- Lee Bergstrand
- Matt McInnes

#### Introduction

- **TODO**

Introduction to SQLAlchemy
==========================

In [58]:
import sqlalchemy

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=False)

### Creating Database-Backed Classes

In [37]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [38]:
from sqlalchemy import Column, Integer, String
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
                self.name, self.fullname, self.password)

In [39]:
Base.metadata.create_all(engine)

##### Issued SQL:

```SQL
CREATE TABLE users (
    id INTEGER NOT NULL, name VARCHAR,
    fullname VARCHAR,
    password VARCHAR,
    PRIMARY KEY (id)
```

### Creating a Session

In [40]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

### Adding and Updating Objects

In [41]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [42]:
our_user = session.query(User).filter_by(name='ed').first()

##### Issued SQL:

*Insert (Flush)*

```SQL
INSERT INTO users (name, fullname, password) VALUES ('ed', 'Ed Jones', 'edspassword')
```

*Query Select*

```SQL
SELECT users.id AS users_id,
        users.name AS users_name,
        users.fullname AS users_fullname,
        users.password AS users_password
FROM users
WHERE users.name = 'ed'
 LIMIT 1 OFFSET 0
```

In [43]:
our_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

In [44]:
ed_user is our_user

True

In [45]:
session.add_all([
        User(name='wendy', fullname='Wendy Williams', password='foobar'),
        User(name='mary', fullname='Mary Contrary', password='xxg527'),
        User(name='fred', fullname='Fred Flinstone', password='blah')])

In [46]:
ed_user.password = 'f8s7ccs'

In [47]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>])

In [48]:
session.new 

IdentitySet([<User(name='mary', fullname='Mary Contrary', password='xxg527')>, <User(name='wendy', fullname='Wendy Williams', password='foobar')>, <User(name='fred', fullname='Fred Flinstone', password='blah')>])

In [49]:
session.commit()

##### Issued SQL:

*Flush remaining changes (inserts and updates)*

```SQL
UPDATE users SET password='f8s7ccs' WHERE users.id = 1
INSERT INTO users (name, fullname, password) VALUES ('wendy', 'Wendy Williams', 'foobar')
INSERT INTO users (name, fullname, password) VALUES ('mary', 'Mary Contrary', 'xxg527')
INSERT INTO users (name, fullname, password) VALUES (fred', 'Fred Flinstone', 'blah')
```

*Finally commit changes*

```SQL
COMMIT
```

In [50]:
ed_user.id

1

### Rolling Back

In [51]:
ed_user.name = 'Edwardo'

In [52]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

In [53]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

[<User(name='Edwardo', fullname='Ed Jones', password='f8s7ccs')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

##### Issued SQL:

*Update 'ed' and insert 'fakeuser' (flush)*

```SQL

UPDATE users SET name='Edwardo' WHERE users.id = 1

INSERT INTO users (name, fullname, password) VALUES ('fakeuser', 'Invalid', '12345')

```

*Query Select*

```SQL
SELECT users.id AS users_id,
        users.name AS users_name,
        users.fullname AS users_fullname,
        users.password AS users_password
FROM users
WHERE users.name IN ('Edwardo', 'fakeuser')
```

In [54]:
session.rollback()

##### Issued SQL:

```SQL
ROLLBACK
```

In [55]:
ed_user.name

u'ed'

In [56]:
fake_user in session

False

In [57]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

[<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>]

##### Issued SQL:

```SQL
SELECT users.id AS users_id,
        users.name AS users_name,
        users.fullname AS users_fullname,
        users.password AS users_password
FROM users
WHERE users.name IN ('ed', 'fakeuser')
```

### Querying